# Classification with ConvNeXt and OpenVINO

The ConvNeXt model is based on the [A ConvNet for the 2020s](https://arxiv.org/abs/2201.03545) paper. The outcome of this exploration is a family of pure ConvNet models dubbed ConvNeXt. Constructed entirely from standard ConvNet modules, ConvNeXts compete favorably with Transformers in terms of accuracy and scalability, achieving 87.8% ImageNet top-1 accuracy and outperforming Swin Transformers on COCO detection and ADE20K segmentation, while maintaining the simplicity and efficiency of standard ConvNets.
The `torchvision.models` subpackage [contains](https://pytorch.org/vision/main/models/convnext.html) several pretrained ConvNeXt model. In this tutorial we will use ConvNeXt Tiny model.

### Table of content:
- [Prerequisites](#Prerequisites)
- [Get a test image](#Get-a-test-image)
- [Get a pretrained model.](#Get-a-pretrained-model)
- [Define a preprocessing and prepare an input data](#Define-a-preprocessing-and-prepare-an-input-data)
- [Use the original model to run an inference](#Use-the-original-model-to-run-an-inference)
- [Convert the model to OpenVINO Intermediate representation format](#Convert-the-model-to-OpenVINO-Intermediate-representation-format)
- [Use the OpenVINO IR model to run an inference](#Use-the-OpenVINO-IR-model-to-run-an-inference)

## Prerequisites

In [ ]:
%pip install -q "torch==2.0.1" "torchvision==0.15.2"
%pip install -q  "openvino>=2023.1.0"

## Get a test image
First of all lets get a test image from an open dataset. 

In [ ]:
from datasets import load_dataset


dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

display(image)

## Get a pretrained model
To get a pretrained model just use a specific method of `torchvision.models`for this model.

In [ ]:
import torchvision.models as models


model = models.convnext_tiny(pretrained=True)

## Define a preprocessing and prepare an input data
Use `torchvision.transforms` to make a preprocessing.

In [ ]:
import torch
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


input_data = transform(image)
input_data = torch.stack([input_data], dim=0)

## Use the original model to run an inference

In [ ]:
outputs = model(input_data)

And print results

In [ ]:
import sys 

sys.path.append("../utils")
from notebook_utils import download_file


# download class number to class label mapping
imagenet_classes_file_path = download_file("https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/datasets/imagenet/imagenet_2012.txt")
imagenet_classes = open(imagenet_classes_file_path).read().splitlines()


def print_results(outputs: torch.Tensor):
    _, predicted_class = outputs.max(1)
    predicted_probability = torch.softmax(outputs, dim=1)[0, predicted_class].item()

    print(f"Predicted Class: {predicted_class.item()}")
    print(f"Predicted Label: {imagenet_classes[predicted_class.item()]}")
    print(f"Predicted Probability: {predicted_probability}")

In [ ]:
print_results(outputs)

## Convert the model to OpenVINO Intermediate representation format

OpenVINO supports PyTorch through conversion to OpenVINO Intermediate Representation (IR) format. To take the advantage of OpenVINO optimization tools and features, the model should be converted using the OpenVINO Converter tool (OVC). The `openvino.convert_model` function provides Python API for OVC usage. The function returns the instance of the OpenVINO Model class, which is ready for use in the Python interface. However, it can also be saved on disk using `openvino.save_model` for future execution.

Starting from OpenVINO 2023.0.0 release OpenVINO supports direct conversion PyTorch models. To perform conversion, we should provide PyTorch model instance and example input into `openvino.convert_model`. For more details, refer to [OpenVINO documentation](https://docs.openvino.ai/2023.1/openvino_docs_model_processing_introduction.html#).

Starting from OpenVINO 2023.0.1 release `torchvision.transforms` were added to OpenVINO preprocessing converter. 

In [ ]:
from pathlib import Path

import numpy as np
import openvino as ov 
from openvino.preprocess.torchvision.preprocess_converter import PreprocessConverter


ov_model_xml_path = Path('models/ov_convnext_model.xml')
input_image = np.expand_dims(np.array(image), axis=0)  # it should be a batch

if not ov_model_xml_path.exists():
    ov_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
    converted_model = ov.convert_model(model, example_input=torch.randn(1, 3, 224, 224))
    # add transform to OpenVINO preprocessing converting
    ov_model = PreprocessConverter.from_torchvision(
        model=converted_model,
        transform=transform,
        input_example=input_image  # you can pass an input image as an input example or generate it
    )
    ov.save_model(converted_model, ov_model_xml_path)
else:
    print(f"IR model {ov_model_xml_path} already exists.")

When the `openvino.save_model` function is used, an OpenVINO model is serialized in the file system as two files with `.xml` and `.bin` extensions. This pair of files is called OpenVINO Intermediate Representation format (OpenVINO IR, or just IR) and useful for efficient model deployment. OpenVINO IR can be loaded into another application for inference using the `openvino.Core.read_model` function. 

In [ ]:
core = ov.Core()

compiled_model = core.compile_model(ov_model_xml_path, device_name='CPU')

## Use the OpenVINO IR model to run an inference

In [ ]:
outputs = compiled_model(input_data)[0]
print_results(torch.from_numpy(outputs))